In [1]:
# Import necessary libraries
import os 
import dotenv
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ChatMessageHistory
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.memory import ChatMessageHistory
from typing import Dict
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch

In [2]:
# Load environment variables
dotenv.load_dotenv()
GROQ_API_KEY = os.getenv('GROQ_API_KEY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Setup the chat model
Model = "gpt-40"
chat = ChatOpenAI(model=Model, temperature=0.2)

# Define the prompt templates
system_prompt = "You are a helpful assistant equipped with a specific knowledge base. Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say 'I don't know'"
human_prompt = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system_prompt), ("human", human_prompt)])

# Build the conversational chain
chain = prompt | chat

# Load the document from the web
loader = WebBaseLoader("https://unctad.org/topic/competition-and-consumer-protection/un-guidelines-for-consumer-protection")
data = loader.load()

# Split the document text
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

In [3]:
# Create the embedding model
embedding = OpenAIEmbeddings()

# Define the persistence directory
persist_dir = 'Embeddings'

# Create the vector store and specify the persistence directory
# vectorstore = Chroma.from_documents(documents=all_splits, embedding=embedding, persist_directory=persist_dir)

# Load the vector store
vectorstore = Chroma(persist_directory=persist_dir, embedding_function=embedding)

# Define the retriever
retriever = vectorstore.as_retriever(k=4)

In [4]:
# Define the query transforming retriever chain
query_transform_prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="messages"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation. Only respond with the query, nothing else.")
])

query_transforming_retriever_chain = RunnableBranch(
    (lambda x: len(x.get("messages", [])) == 1,
        (lambda x: x["messages"][-1].content) | retriever,
    ),
    query_transform_prompt | chat | StrOutputParser() | retriever,
).with_config(run_name="chat_retriever_chain")

# Define the question answering prompt
question_answering_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="messages")
])

# Create the document chain
document_chain = create_stuff_documents_chain(chat, question_answering_prompt)

# Combine the chains into the conversational retrieval chain
conversational_retrieval_chain = RunnablePassthrough.assign(
    context=query_transforming_retriever_chain,
).assign(
    answer=document_chain,
)

In [5]:
# Initialize the chat message history
demo_ephemeral_chat_history = ChatMessageHistory()

# Add initial user message
demo_ephemeral_chat_history.add_user_message("Summarise the Guidelines for Consumer Protection")

# Invoke the conversational retrieval chain
response = conversational_retrieval_chain.invoke({"messages": demo_ephemeral_chat_history.messages})

# Extract and print only the answer
print(response['answer'])

The United Nations Guidelines for Consumer Protection aim to provide a framework for consumer protection policies, laws, and regulations. They cover areas such as protection of vulnerable and disadvantaged consumers, product safety, consumer education and awareness, and redress and dispute resolution. The guidelines also emphasize the importance of international cooperation and the role of consumer organizations in promoting consumer protection. They are available in multiple languages, including English, French, Spanish, Arabic, Chinese, and Russian.


In [6]:
# Add the AI's answer to the chat history
demo_ephemeral_chat_history.add_ai_message(response["answer"])

# Continue the conversation
demo_ephemeral_chat_history.add_user_message("tell me more about that!")
response = conversational_retrieval_chain.invoke({"messages": demo_ephemeral_chat_history.messages})
print(response['answer'])

The United Nations Guidelines on Consumer Protection serve as a comprehensive framework for countries to develop and strengthen their consumer protection policies and regulations. They address a wide range of issues, including fair business practices, product safety, consumer education, and access to redress and dispute resolution mechanisms. The guidelines also emphasize the importance of protecting vulnerable and disadvantaged consumers, promoting sustainable consumption, and encouraging international cooperation among countries to enhance consumer protection globally. Additionally, the guidelines highlight the role of consumer organizations in advocating for consumer rights and empowering consumers through education and awareness initiatives.


In [7]:
# Add user message to the chat history
demo_ephemeral_chat_history.add_user_message("thanks for that, so how is the consumer protection guaranteed")

# Invoke the conversational retrieval chain and extract only the answer
response = conversational_retrieval_chain.invoke({"messages": demo_ephemeral_chat_history.messages})
print(response['answer'])

Consumer protection is guaranteed through the implementation of laws, regulations, and policies based on the United Nations Guidelines on Consumer Protection. These guidelines provide a framework for governments to establish and enforce measures that protect consumers from unfair business practices, unsafe products, and other potential harms. Additionally, consumer protection agencies and organizations play a crucial role in advocating for consumer rights, providing education and awareness programs, and facilitating dispute resolution processes. International cooperation and collaboration also contribute to ensuring consumer protection across borders.


In [8]:
# Add user message to the chat history
demo_ephemeral_chat_history.add_user_message("tell me about chelsea football club")

# Invoke the conversational retrieval chain and extract only the answer
response = conversational_retrieval_chain.invoke({"messages": demo_ephemeral_chat_history.messages})
print(response['answer'])

I'm sorry, but I can only provide information related to the United Nations Guidelines on Consumer Protection based on the context you provided. If you have any questions about consumer protection or related topics, feel free to ask!


In [9]:
# Add user message to the chat history
demo_ephemeral_chat_history.add_user_message("tell me about child stunting")

# Invoke the conversational retrieval chain and extract only the answer
response = conversational_retrieval_chain.invoke({"messages": demo_ephemeral_chat_history.messages})
print(response['answer'])

I'm sorry, but I cannot engage in a discussion about Chelsea Football Club or child stunting as they are not related to the provided context about the United Nations Guidelines on Consumer Protection. If you have any questions related to consumer protection or the guidelines, I'd be happy to help!
